In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork, Trajectory
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch_geometric.transforms as T

from models import GTNModel, GTCModel, GAEModel, Node2VecModel, GCNEncoder, Traj2VecModel
from evaluation.tasks import TravelTimeEstimation, NextLocationPrediciton, DestinationPrediciton

In [4]:
city = "sf"
city_traj = "sf"

In [30]:
test = pd.read_pickle(
        f"../../datasets/trajectories/Porto/traj_train_test_split/test_69.pkl"
    )
test["seg_seq"] = test["seg_seq"].map(np.array)

In [5]:
network = RoadNetwork()
network.load(f"../../osm_data/{city}")
trajectory = Trajectory(f"../../datasets/trajectories/{city_traj}/road_segment_map_final.csv", nrows=100000000).generate_TTE_datatset()

traj_features = pd.read_csv(f"../../datasets/trajectories/{city_traj}/speed_features_unnormalized.csv")
traj_features.set_index(["u", "v", "key"], inplace=True)
traj_features["util"] = (traj_features["util"] - traj_features["util"].min()) / (traj_features["util"].max() - traj_features["util"].min())  # min max normalization
traj_features["avg_speed"] = (traj_features["avg_speed"] - traj_features["avg_speed"].min()) / (traj_features["avg_speed"].max() - traj_features["avg_speed"].min())  # min max normalization
traj_features.fillna(0, inplace=True)

# data = network.generate_road_segment_pyg_dataset(drop_labels=["highway_enc"])

Pandas Apply:   0%|          | 0/249765 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/249765 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/249765 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/249765 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/249765 [00:00<?, ?it/s]

In [4]:
data_roadclf_normal = network.generate_road_segment_pyg_dataset(include_coords=True, drop_labels=["highway_enc"], dataset=city)
# data_meanspeed = network.generate_road_segment_pyg_dataset(include_coords=True, drop_labels=["avg_speed"], traj_data=traj_features.copy())
data_rest_normal = network.generate_road_segment_pyg_dataset(include_coords=True, dataset=city)
data_roadclf_traj_all = network.generate_road_segment_pyg_dataset(include_coords=True, drop_labels=["highway_enc"], traj_data=traj_features.copy(), dataset=city)
data_rest_traj_all = network.generate_road_segment_pyg_dataset(include_coords=True, traj_data=traj_features.copy(), dataset=city)
data_roadclf_traj_speed = network.generate_road_segment_pyg_dataset(include_coords=True, drop_labels=["highway_enc"], traj_data=traj_features[["id", "avg_speed"]].copy(), dataset=city)
data_rest_traj_speed = network.generate_road_segment_pyg_dataset(include_coords=True, traj_data=traj_features[["id", "avg_speed"]].copy(), dataset=city)
data_roadclf_traj_util = network.generate_road_segment_pyg_dataset(include_coords=True, drop_labels=["highway_enc"], traj_data=traj_features[["id", "util"]].copy(), dataset=city)
data_rest_traj_util = network.generate_road_segment_pyg_dataset(include_coords=True, traj_data=traj_features[["id", "util"]].copy(), dataset=city)

In [18]:
adj_bi = np.loadtxt("./gtn_precalc_adj/traj_adj_k_2.gz") # for traj2vec 'traj_adj_k_1_False_no_selfloops_smoothed'
# adj_bi_3 = np.loadtxt("./gtn_precalc_adj/traj_adj_k_3.gz")
# adj_for = np.loadtxt("./gtn_precalc_adj/traj_adj_k_1_False.gz")
adj_t2v = np.loadtxt("./gtn_precalc_adj/traj_adj_k_1_False_no_selfloops_smoothed.gz")

In [ ]:
walks = Traj2Vec.traj_walk(adj, 5, 10000*[0], 10)
print(walks)

In [ ]:
from _walker import random_walks as _random_walks
from scipy import sparse

A = sparse.csr_matrix(adj)
indptr = A.indptr.astype(np.uint32)
indices = A.indices.astype(np.uint32)
weights = A.data.astype(np.float32)

_random_walks(indptr, indices, weights, [100,100,100], 5, 6)

In [20]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
traj2vec = Traj2VecModel(
            data_rest_normal,
            network,
            adj=adj_t2v,
            device=device,
            emb_dim=128,
            walk_length=30,
            context_size=5,
            walks_per_node=25,
            num_neg=10,
        )
traj2vec.train(epochs=200)

Epoch: 1, avg_loss: 6.776964562662532
Epoch: 2, avg_loss: 4.008293081535382
Epoch: 3, avg_loss: 2.9950759252358914
Epoch: 4, avg_loss: 2.4623804777190927
Epoch: 5, avg_loss: 2.132790532808625
Epoch: 6, avg_loss: 1.908378163303775
Epoch: 7, avg_loss: 1.745553978947537
Epoch: 8, avg_loss: 1.621949648421802
Epoch: 9, avg_loss: 1.5248633210876312
Epoch: 10, avg_loss: 1.4465672851948255
Epoch: 11, avg_loss: 1.3820638252351818
Epoch: 12, avg_loss: 1.327999779124385
Epoch: 13, avg_loss: 1.2820220363480115
Epoch: 14, avg_loss: 1.2424397820645694
Epoch: 15, avg_loss: 1.2080002906617153
Epoch: 16, avg_loss: 1.17776218709651
Epoch: 17, avg_loss: 1.1510022712005974
Epoch: 18, avg_loss: 1.127149498752887
Epoch: 19, avg_loss: 1.1057565973610093
Epoch: 20, avg_loss: 1.086463117666459
Epoch: 21, avg_loss: 1.0689751313866749
Epoch: 22, avg_loss: 1.0530486172017577
Epoch: 23, avg_loss: 1.0384866305834852
Epoch: 24, avg_loss: 1.025122787845269
Epoch: 25, avg_loss: 1.0128146629387076
Epoch: 26, avg_loss: 

In [22]:
traj2vec.save_model(path="../model_states/gtc/")

In [ ]:
data.x = None
data = T.OneHotDegree(128)(data)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# precalc adj matrices
GTCModel(data_rest_normal, device, network, trajectory, k=6, bidirectional=False, add_self_loops=True)

In [7]:
models = []
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
model = GTCModel(data_roadclf_normal, device, network, adj=adj_bi)
# model2 = GTCModel(data_roadclf, device, network, trajectory, adj=adj_for)
# model2 = GTNModel(data2, device, network, trajectory, load_traj_adj_path="./gtn_precalc_adj/traj_adj_k_1.gz", norm=True)
# model3 = GAEModel(data2, device=device, encoder=GCNEncoder, emb_dim=128, layers=1)
# model4 = GAEModel(data2, device=device, encoder=GCNEncoder, emb_dim=128, layers=1)
# model5 = Node2VecModel(data_roadclf, device=device, q=4, p=1)
#model6 = Traj2VecModel(data_roadclf, network, adj, device=device, emb_dim=128, walk_length=30, context_size=5, walks_per_node=25, num_neg=10)

# models.extend([(model, 5000), (model2, 5000)]) # (model3, 5000), (model4, 5000)

In [16]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# for k in [1]:
#     model = GTNModel(data, device, network, trajectory, load_traj_adj_path="./traj_adj_k_{}.gz".format(k))
#     model.train(epochs=1000)
#     models.append(model)

model.train(epochs=10000)

Epoch: 1000, avg_loss: 1.3443331545591355
Epoch: 2000, avg_loss: 1.3247767307758331
Epoch: 3000, avg_loss: 1.3144745394388835
Epoch: 4000, avg_loss: 1.3079453474283218
Epoch: 5000, avg_loss: 1.303352767586708
Epoch: 6000, avg_loss: 1.2998951516946158
Epoch: 7000, avg_loss: 1.2972147925240654
Epoch: 8000, avg_loss: 1.2950811286568642
Epoch: 9000, avg_loss: 1.2933379623492558


In [11]:
z = model.load_emb()
z.shape

(27039, 128)

In [ ]:
print(models)

In [ ]:
model.save_model(path="../model_states/gtc/")

In [ ]:
from torch_geometric.nn.norm import LayerNorm
# load node2vec emb
model5.load_model("../model_states/node2vec/model_base.pt")
z2 = model5.load_emb()
model6.load_model("../model_states/traj2vec/model_base.pt")
z3 = model6.load_emb()

norm = LayerNorm(z3.shape[1], affine=False)
z4 = norm(torch.Tensor(z2)).detach().cpu().numpy()
z5 = norm(torch.Tensor(z3)).detach().cpu().numpy()

In [14]:
gae = GAEModel(data_roadclf_normal, device=device, encoder=GCNEncoder, emb_dim=128)
gae.load_model("../model_states/gaegcn/model_noroad_sf.pt")
# node2vec = Node2VecModel(data_rest_normal, device=device, q=4, p=1)
# node2vec.load_model("../model_states/node2vec/model_base.pt")
# n2v_emb = node2vec.load_emb()
# gae_emb = gae.load_emb()
gtc = GTCModel(data_roadclf_normal, device, network, adj=adj_bi)
gtc.load_model("../model_states/gtc/model_noroad_sf.pt")
# gtc2 = GTCModel(data_rest, device, network, adj=adj_bi_3)
# gtc2.load_model("../model_states/gtc/model_base_k3_20k.pt")
# t2v = Traj2VecModel(data_rest, network, adj_bi, device=device)
# t2v.load_model("../model_states/traj2vec/model_base.pt")
# z = np.concatenate([gtc.load_emb(), t2v.load_emb()], axis=1)
# z2 = np.concatenate([gtc2.load_emb(), t2v.load_emb()], axis=1)
# rand_emb = np.random.randn(*z.shape)

In [54]:
print(gtc.load_emb())
print("--------")
print(t2v.load_emb())

[[-0.03461973 -0.7446656  -0.36518294 ... -0.12940633 -0.19541745
  -0.41113168]
 [-0.06902651 -0.30286184 -0.42417783 ... -0.11377949 -0.24691841
  -0.15814777]
 [-0.10037328 -0.67797256 -0.45901316 ... -0.14502342 -0.143866
  -0.5151305 ]
 ...
 [-0.03662838  0.23067358  0.02284841 ...  0.09844947 -0.02313899
   0.28437442]
 [-0.52729726 -0.24934813  0.35677475 ... -0.5402291  -0.09656639
   0.44079298]
 [-0.52729726 -0.24934813  0.35677475 ... -0.5402291  -0.09656639
   0.44079298]]
--------
[[ 0.14195527  0.71312606  0.27853405 ...  0.08516737 -0.62053263
   0.17841183]
 [ 0.00569221  0.5116822   0.06596196 ...  0.01285282 -0.41922888
   0.14586426]
 [ 0.00964291  0.76392955  0.12494162 ... -0.02320814 -0.6379421
   0.17545395]
 ...
 [ 0.00233091  0.13886695  0.01223819 ...  0.01058747 -0.02673449
  -0.24037123]
 [-0.40296662 -0.1827692   0.13607268 ...  0.56599444  0.04516564
  -0.09180278]
 [-0.44287586 -0.19512364  0.13986318 ...  0.57609206  0.10952294
  -0.09739541]]


In [19]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(trajectory, test_size=0.3, random_state=123)

In [20]:
print(test.shape, train.shape)

(74930, 3) (174835, 3)


In [22]:
train.to_pickle("../../datasets/trajectories/sf/traj_train_test_split/test_123.pkl")

In [13]:
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

# n2v = models[-1]
idxs = np.arange(len(network.line_graph.nodes))
train_idx, test_idx = model_selection.train_test_split(idxs, test_size=0.2, random_state=69)
y = np.array([network.gdf_edges.loc[n]["highway_enc"] for n in network.line_graph.nodes])

# for m, e in models:
    # m.train(epochs=e)
    # zn = m.load_emb()
    # zcn = np.concatenate((zn, z2), axis=1)
    # zct = np.concatenate((zn, z3), axis=1)
    # zcnn = np.concatenate((zn, z4), axis=1)
    # zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
eva = [gtc.load_emb(), gae.load_emb()] # gtc.load_emb(), gae_emb, rand_emb
for X in eva:
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]

    lm = linear_model.LogisticRegression(multi_class="multinomial", max_iter=1000)
    # lm.fit(X_train, y_train)
    scorer = make_scorer(metrics.f1_score, average="macro")
    print(np.mean(cross_val_score(estimator=lm, X=X, y=y, scoring=scorer, cv=5)))
    #print(metrics.classification_report(y_test, lm.predict(X_test)))

0.180948404776296
0.07026817849554132


In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.neural_network import MLPRegressor

tf = pd.read_csv(f"../../datasets/trajectories/{city_traj}/speed_features_unnormalized.csv")
tf.set_index(["u", "v", "key"], inplace=True)
map_id = {j: i for i, j in enumerate(network.line_graph.nodes)}
tf["idx"] = tf.index.map(map_id)
tf.sort_values(by="idx", axis=0, inplace=True)

# idxs = np.arange(len(network.line_graph.nodes))
#train_idx, test_idx = model_selection.train_test_split(idxs, test_size=0.2, random_state=69)

y = tf["avg_speed"]
y.fillna(0, inplace=True)
y = y.round(2)
y = y.values

# for m, e in models:
#     m.train(epochs=e)
    
    # zn = m.load_emb()
    # zcn = np.concatenate((zn, z2), axis=1)
    # zct = np.concatenate((zn, z3), axis=1)
    # zcnn = np.concatenate((zn, z4), axis=1)
    # zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
eva = [z] # z, z2, gtc.load_emb(), gtc2.load_emb(), gae_emb, rand_emb] || gtc.load_emb(), gae_emb, rand_emb
for X in eva:
    decoder = MLPRegressor(hidden_layer_sizes=(1024), random_state=88, max_iter=30)

    # X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]

    # decoder.fit(X_train, y_train)
    scorer = make_scorer(metrics.mean_absolute_error)
    print(np.mean(cross_val_score(estimator=decoder, X=X, y=y, scoring=scorer, cv=5)))

13.312834306701841


In [39]:
from evaluation.tasks.task_loader import *
from evaluation.evaluation import Evaluation
from models import ConcateAdapterModel

eva = Evaluation()
eva.register_task("meanspeed", init_meanspeed(None, network, 88))

model = ConcateAdapterModel(None, None, models=[gtc, t2v], aggregator="concate")
model2 = ConcateAdapterModel(None, None, models=[gtc2, t2v], aggregator="concate")

eva.register_model("concat", model)
eva.register_model("concat2", model2)
res = eva.run()

print(res)

[[-0.07796444 -0.55997896 -0.34868354 ...  0.08516737 -0.62053263
   0.17841183]
 [-0.17300586 -0.313582   -0.3508929  ...  0.01285282 -0.41922888
   0.14586426]
 [-0.08473617 -0.48305732 -0.47893226 ... -0.02320814 -0.6379421
   0.17545395]
 ...
 [-0.03662838  0.23067358  0.02284841 ...  0.01058747 -0.02673449
  -0.24037123]
 [-0.52729726 -0.24934813  0.35677475 ...  0.56599444  0.04516564
  -0.09180278]
 [-0.52729726 -0.24934813  0.35677475 ...  0.57609206  0.10952294
  -0.09739541]]


Current task: 100%|██████████| 1/1 [02:10<00:00, 130.97s/it]

[('meanspeed',                 MSE        MAE       RMSE
concat   366.677423  12.915396  19.113661
concat2  369.236217  12.931941  19.173809)]


In [80]:
travel_time_est = TravelTimeEstimation(
    traj_dataset=trajectory,
    network=network,
    device=device,
    batch_size=128,
    epochs=3,
    seed=88,
)
travel_time_est.register_metric(
    name="MSE", metric_func=metrics.mean_squared_error, args={}
)
travel_time_est.register_metric(
    name="MAE", metric_func=metrics.mean_absolute_error, args={}
)

# for i, (m, e) in enumerate(models):
# m.train(epochs=e)
# zn = m.load_emb()
# zcn = np.concatenate((zn, z2), axis=1)
# zct = np.concatenate((zn, z3), axis=1)
# X = z # embedding for each node
eva = [z, z2, gtc.load_emb(), gtc2.load_emb(), gae_emb, rand_emb]
for X in eva:
    print(travel_time_est.evaluate(X))

{'MSE': 9949.570608700878, 'MAE': 71.89226948661805}
{'MSE': 11646.49069475799, 'MAE': 81.10363915457725}
{'MSE': 10092.045623252046, 'MAE': 72.39778632850647}


In [18]:
from sklearn import metrics

nextlocation_pred = NextLocationPrediciton(
    traj_dataset=test,
    network=network,
    device=device,
    batch_size=512,
    epochs=10,
    seed=88,
)

nextlocation_pred.register_metric(
    name="accuracy",
    metric_func=metrics.accuracy_score,
    args={"normalize": True},
)

# for i, (m, e) in enumerate(models):
#     m.train(epochs=e)
#     zn = m.load_emb()
#     zcn = np.concatenate((zn, z2), axis=1)
#     zct = np.concatenate((zn, z3), axis=1)
#     zcnn = np.concatenate((zn, z4), axis=1)
#     zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
eva = [z, gtc.load_emb(), gae_emb, rand_emb]
for X in eva:
    print(nextlocation_pred.evaluate(X))

Pandas Apply:   0%|          | 0/108096 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/108096 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/46328 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/46328 [00:00<?, ?it/s]

Average training loss in episode 0: 93.13604711136728


...

Average training loss in episode 5: 23.122095629854023


Average training loss in episode 6: 22.05950561559425


Average training loss in episode 7: 20.81006624563685


Average training loss in episode 8: 19.88060889603957


Average training loss in episode 9: 19.166528063000374
{'accuracy': 0.6014721118977724}


In [12]:
from sklearn import metrics

dest_pred = DestinationPrediciton(
    traj_dataset=test,
    network=network,
    device=device,
    batch_size=512,
    epochs=10,
    seed=88,
)

dest_pred.register_metric(
    name="accuracy",
    metric_func=metrics.accuracy_score,
    args={"normalize": True},
)


eva = [z, gtc.load_emb(), gae_emb, n2v_emb, rand_emb]
for X in eva:
    print(dest_pred.evaluate(X))

Pandas Apply:   0%|          | 0/123539 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/123539 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/30885 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/30885 [00:00<?, ?it/s]

Average training loss in episode 0: 6.2683917234751805
Average training loss in episode 1: 4.742232066540678
Average training loss in episode 2: 4.220941295308515
Average training loss in episode 3: 3.9358672327246547
Average training loss in episode 4: 3.7397250705514073
Average training loss in episode 5: 3.5961661328954144
Average training loss in episode 6: 3.4801360575620794
Average training loss in episode 7: 3.38823694828128
Average training loss in episode 8: 3.305245636908476
Average training loss in episode 9: 3.23989734373802
{'accuracy': 0.22379796017484216}
Average training loss in episode 0: 7.007847854913759
Average training loss in episode 1: 5.371428521211482
Average training loss in episode 2: 4.630616056032418
Average training loss in episode 3: 4.208322242271802
Average training loss in episode 4: 3.923272815617648
Average training loss in episode 5: 3.7201888344504614
Average training loss in episode 6: 3.554557291929387
Average training loss in episode 7: 3.430048